In [0]:
%run /Workspace/Users/joao.vitormsilva@usp.br/vendas_livraria/vendas_livraria



# Imports, inicialização dos logs e Workspace Client

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


# Funções de qualidade de dados

# Lendo a tabela criada

+----------+--------+--------------+--------+---------+--------------------+----------+--------------------+------+
|data_venda|id_venda|id_linha_venda|id_livro|qtde_item|          nome_livro| categoria|               autor| preco|
+----------+--------+--------------+--------+---------+--------------------+----------+--------------------+------+
|2025-02-04|    4073|          4403|       5|        2|        Dom Casmurro|    Ficção|    Machado de Assis| 86.81|
|2024-08-25|    5136|          5466|       5|        4|        Dom Casmurro|    Ficção|    Machado de Assis|131.48|
|2024-12-18|    NULL|          4485|       2|        5|                1984|    Ficção|       George Orwell| 92.92|
|2025-05-08|    2451|          2781|       9|        2|A Revolução dos B...|    Ficção|       George Orwell|139.69|
|2024-12-29|    4615|          4945|       1|        1|A Menina que Roub...|    Ficção|        Markus Zusak| 92.76|
|2025-02-27|    2825|          3155|       8|        5|  O Pequeno Prínc

...


# Testes de qualidade na tabela utilizando o DQX

id_venda: {'count': 1000, 'mean': 4837.601214574899, 'stddev': 2842.488875969202, 'min': 1, '25%': 2537, '50%': 4887, '75%': 7294, 'max': 9666, 'count_non_null': 988, 'count_null': 12}
id_linha_venda: {'count': 1000, 'mean': 5145.453, 'stddev': 2849.663524346865, 'min': 2, '25%': 2857, '50%': 5195, '75%': 7591, 'max': 9996, 'count_non_null': 1000, 'count_null': 0}
id_livro: {'count': 1000, 'mean': 4.926, 'stddev': 2.5598978645791823, 'min': 1, '25%': 3, '50%': 5, '75%': 7, 'max': 9, 'count_non_null': 1000, 'count_null': 0}
qtde_item: {'count': 1000, 'mean': 2.968, 'stddev': 1.4173866744770054, 'min': 1, '25%': 2, '50%': 3, '75%': 4, 'max': 5, 'count_non_null': 1000, 'count_null': 0}
nome_livro: {'count': 1000, 'mean': 1984.0, 'stddev': 0.0, 'min': '1984', '25%': '1984.0', '50%': '1984.0', '75%': '1984.0', 'max': 'Sapiens', 'count_non_null': 1000, 'count_null': 0}
categoria: {'count': 1000, 'mean': None, 'stddev': None, 'min': 'Ficção', '25%': None, '50%': None, '75%': None, 'max': 'Não

{'check': {'function': 'is_not_null', 'arguments': {'column': 'data_venda'}}, 'name': 'data_venda_is_null', 'criticality': 'error'}
{'check': {'function': 'is_in_range', 'arguments': {'column': 'id_venda', 'min_limit': 1, 'max_limit': 9666}}, 'name': 'id_venda_isnt_in_range', 'criticality': 'error'}
{'check': {'function': 'is_not_null', 'arguments': {'column': 'id_linha_venda'}}, 'name': 'id_linha_venda_is_null', 'criticality': 'error'}
{'check': {'function': 'is_in_range', 'arguments': {'column': 'id_linha_venda', 'min_limit': 2, 'max_limit': 9996}}, 'name': 'id_linha_venda_isnt_in_range', 'criticality': 'error'}
{'check': {'function': 'is_not_null', 'arguments': {'column': 'id_livro'}}, 'name': 'id_livro_is_null', 'criticality': 'error'}
{'check': {'function': 'is_in_list', 'arguments': {'column': 'id_livro', 'allowed': [5, 1, 8, 2, 6, 3, 4, 9, 7]}}, 'name': 'id_livro_other_value', 'criticality': 'error'}
{'check': {'function': 'is_in_range', 'arguments': {'column': 'id_livro', 'min_

+----------+--------+--------------+--------+---------+--------------------+---------+----------------+------+
|data_venda|id_venda|id_linha_venda|id_livro|qtde_item|          nome_livro|categoria|           autor| preco|
+----------+--------+--------------+--------+---------+--------------------+---------+----------------+------+
|2025-02-04|    4073|          4403|       5|        2|        Dom Casmurro|   Ficção|Machado de Assis| 86.81|
|2024-08-25|    5136|          5466|       5|        4|        Dom Casmurro|   Ficção|Machado de Assis|131.48|
|2024-12-18|    NULL|          4485|       2|        5|                1984|   Ficção|   George Orwell| 92.92|
|2025-05-08|    2451|          2781|       9|        2|A Revolução dos B...|   Ficção|   George Orwell|139.69|
|2024-12-29|    4615|          4945|       1|        1|A Menina que Roub...|   Ficção|    Markus Zusak| 92.76|
+----------+--------+--------------+--------+---------+--------------------+---------+----------------+------+



+----------+--------+--------------+--------+---------+----------+---------+-----+-----+
|data_venda|id_venda|id_linha_venda|id_livro|qtde_item|nome_livro|categoria|autor|preco|
+----------+--------+--------------+--------+---------+----------+---------+-----+-----+
|         0|     101|             0|       0|        0|         0|        0|    0|    0|
+----------+--------+--------------+--------+---------+----------+---------+-----+-----+



In [0]:
import unittest
import logging
from pyspark.sql import functions as F

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class TestReadFile(unittest.TestCase):

    def test_read_file(self):
        logger.info("Teste de leitura de arquivo")

        df_inicial = spark.table('workspace.bronze.vendas_livros')

        logger.info(f'Tamanho do arquivo: {df_inicial.count()}')

        self.assertIsNotNone(df_inicial)
        self.assertGreater(df_inicial.count(), 0)

if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_read_file (__main__.TestReadFile.test_read_file) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.973s

OK


+----------+-------------------+
|data_venda|valor_autor_popular|
+----------+-------------------+
|2024-06-16|                 29|
|2024-06-17|                 23|
|2024-06-18|                 19|
|2024-06-19|                 12|
|2024-06-20|                 19|
+----------+-------------------+



StructType([StructField('data_venda', DateType(), True), StructField('id_venda', LongType(), True), StructField('id_linha_venda', LongType(), True), StructField('id_livro', LongType(), True), StructField('qtde_item', LongType(), True), StructField('nome_livro', StringType(), True), StructField('categoria', StringType(), True), StructField('autor', StringType(), True), StructField('preco', DoubleType(), True)])

+----------+------------+-----------------+-------------------+------------------+------------------+-----------------+-------------------+
|data_venda|total_vendas|quantidade_ficcao|quantidade_n_ficcao|valor_maximo_venda|valor_minimo_venda|valor_medio_venda|valor_autor_popular|
+----------+------------+-----------------+-------------------+------------------+------------------+-----------------+-------------------+
|2024-06-16|          89|               83|                  6|            743.35|             42.72|           213.84|                 29|
|2024-06-17|         127|              104|                 23|             678.1|             34.87|           245.68|                 23|
|2024-06-18|          63|               45|                 18|            746.65|             21.85|           267.73|                 19|
|2024-06-19|          58|               52|                  6|             620.0|             21.73|           210.12|                 12|
|2024-06-20|        

DataFrame[]

George Orwell


2024-06-17


In [0]:
import unittest
import logging
from pyspark.sql import functions as F

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class TestSuiteSchemaTable(unittest.TestCase):

    def test_schema_table(self):
        logger.info("Teste de validação do schema da tabela bronze")

        df_inicial = spark.table('workspace.bronze.vendas_livros')
        expected_schema = {
            "data_venda": "date",
            "id_venda": "bigint",
            "id_linha_venda": "bigint",
            "id_livro": "bigint",
            "qtde_item": "bigint",
            "nome_livro": "string",
            "categoria": "string",
            "autor": "string",
            "preco": "double"
        }
        for field in df_inicial.schema.fields:
            assert field.name in expected_schema
            assert field.dataType.simpleString() == expected_schema[field.name]

    

if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)   

test_read_file (__main__.TestReadFile.test_read_file) ... ok
test_schema_table (__main__.TestSuiteSchemaTable.test_schema_table) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.971s

OK


In [0]:
import unittest
import logging
from pyspark.sql import functions as F

logging.basicConfig(level = logging.INFO)
logger = logging.getLogger(__name__)

class TestSuiteDates(unittest.TestCase):

  def test_count_dates(self):
    logger.info("Teste de validação quantidade miníma de datas")

    df_inicial = spark.table('workspace.bronze.vendas_livros')
    
    df_dates = df_inicial.select(F.col('data_venda')).distinct().orderBy(F.col('data_venda'), ascending = False)

    self.assertGreaterEqual(df_dates.count(), 10, "Não foi possível achar no minímo 10 datas diferentes")
  
  def test_future_dates(self):
    logger.info("Teste de validação de datas futuras")
    
    df_inicial = spark.table('workspace.bronze.vendas_livros')
    
    df_dates = df_inicial.filter(F.col('data_venda') > F.current_date()).count()
  
    self.assertEqual(df_dates, 0, "Existem datas de venda no futuro!")

if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_read_file (__main__.TestReadFile.test_read_file) ... ok
test_count_dates (__main__.TestSuiteDates.test_count_dates) ... ok
test_future_dates (__main__.TestSuiteDates.test_future_dates) ... ok
test_schema_table (__main__.TestSuiteSchemaTable.test_schema_table) ... ok

----------------------------------------------------------------------
Ran 4 tests in 1.753s

OK


In [0]:
import unittest
import logging
from pyspark.sql import functions as F

logging.basicConfig(level = logging.INFO)
logger = logging.getLogger(__name__)

class TestSuiteNegatives(unittest.TestCase):

  def test_negative_id(self):
    logger.info("Teste de ids negativos")

    df_inicial = spark.table('workspace.bronze.vendas_livros')

    negatives = df_inicial.filter(
        (F.col('id_venda') < 0) &
        (F.col('id_linha_venda') < 0) &
        (F.col('id_livro') < 0)
           ).count()

    self.assertEqual(negatives, 0, "Existem ids negativos na tabela")
  
if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_read_file (__main__.TestReadFile.test_read_file) ... ok
test_count_dates (__main__.TestSuiteDates.test_count_dates) ... ok
test_future_dates (__main__.TestSuiteDates.test_future_dates) ... ok
test_negative_id (__main__.TestSuiteNegatives.test_negative_id) ... ok
test_schema_table (__main__.TestSuiteSchemaTable.test_schema_table) ... ok

----------------------------------------------------------------------
Ran 5 tests in 2.013s

OK
